In [ ]:
import os

In [ ]:
np.linspace(1., 0.5, 11)

In [ ]:
list(itertools.product(
    [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    [2. ** (i / 2) for i in range(2)],
    ))[3]

In [ ]:
[0., *[10. ** (i / 2) for i in range(-2, 3)]][1]

In [ ]:
[0., *[10. ** (i / 2) for i in range(-2, 3)]]

In [ ]:
import numpy as np

In [ ]:
10 ** np.linspace(0, 1, 11)

In [ ]:
[(i / 4.) for i in range(8, 29)]

In [ ]:
np.linspace(2.5, 7.5, 21)

In [ ]:
[0., *(float(n) for n in np.round(10 ** np.linspace(-2, 1, 13), 3))]

In [ ]:
[10. ** (i / 2) for i in range(11)][3]

In [ ]:
[0., *[10. ** (i / 2) for i in range(-2, 3)]]

In [ ]:
def get_aspect(f):
    outer = 1 / (1 - f)
    inner = outer - 1
    full = math.pi * (outer + inner)
    cycles = round(full / 2) * 2
    partial = round(full / cycles, 3)
    return cycles, partial
cycles, partial = get_aspect(f)

In [ ]:
[get_aspect(f) for f in np.linspace(0.95, 0.5, 10)]

In [ ]:
[0., *[10. ** (i / 2) for i in range(-2, 3)]][2]

In [ ]:
np.linspace(0.95, 0.5, 10)

In [ ]:
[10. ** (i / 2) for i in range(11)][9]

In [ ]:
31622 / 3e4

In [ ]:
get_aspect(0.1)

In [ ]:
get_aspect(0.9)

In [ ]:
np.round(2 ** np.linspace(0, 0.5, 6), 3)

In [ ]:
np.linspace(1.0, 0.5, 11)

In [ ]:
ARG1, ARG2, ARG3, alpha = 1, 2, 3, 4

In [ ]:
args = [str(a) for a in [ARG1, ARG2, ARG3, round(math.log10(alpha))]]

In [ ]:
args

In [ ]:
'_'.join(args)

In [ ]:
import itertools

import numpy as np

In [ ]:
list(itertools.product(
    [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    [2. ** (i / 2) for i in range(2)],
    ))

In [ ]:
np.linspace(5, 6, 11)

In [ ]:
round(9.35 / 2) * 2

In [ ]:
def get_aspect(f):
    outer = 1 / (1 - f)
    inner = outer - 1
    full = math.pi * (outer + inner)
    cycles = round(full / 2) * 2
    partial = round(full / cycles, 3)
    return cycles, partial
# cycles, partial = get_aspect(f)

In [ ]:
get_aspect(0.5)

In [ ]:
import init

In [ ]:
from crisis_model.array import *
import numpy as np
from functools import partial
import warnings

class Covid1:

    def __init__(self,
            # params
                aspect = 1.2, # x length relative to y length
                scale = 22., # y length in km
                corner = [0., 0.], # coords of bottom-left corner
                timescale = 1., # days per timestep
                popDensity = 508, # people per sq km
                initialIndicated = 1, # initial mystery cases
                directionChange = 0.5, # where 1 == 180 deg per day
                speed = 5, # agent travel speed in km / h
                infectionChance = 0.1, # chance of transmission by 'contact'
                recoverMean = 14, # average recovery time in days
                recoverSpread = 2, # standard deviations of recovery curve
                contactLength = 1.5, # proximity in metres defining 'contact'
                spatialDecimals = None, # spatial precision limit
                seed = 1066, # random seed
            # _configs
                agentCoords = None,
                headings = None,
                indicated = False,
                recovered = False,
                timeIndicated = 0.,
            ):

        super().__init__()

    @staticmethod
    def _construct(p):

        nAgents = int(p.scale ** 2 * p.aspect * p.popDensity)
        travelLength = p.speed * p.timescale * 24.

        minCoords, maxCoords, domainLengths = get_coordInfo(
            p.corner, p.aspect, p.scale
            )

        agentCoords = np.full((nAgents, 2), np.nan, dtype = float)

        headings = np.full(nAgents, np.nan, dtype = float)
        distances = np.full(nAgents, np.nan, dtype = float)

        indicated = np.empty(nAgents, dtype = bool)
        timeIndicated = np.zeros(nAgents, dtype = float)
        recovered = np.empty(nAgents, dtype = bool)
        susceptible = np.empty(nAgents, dtype = bool)

        def iterate_coords(rng):
            distances[...] = rng.random(nAgents) * travelLength
            ang = p.directionChange * p.timescale
            headings[...] += (rng.random(nAgents) - 0.5) * 2. * np.pi * ang
            wrap = minCoords, maxCoords
            displace_coords(agentCoords, distances, headings, wrap)
            if not p.spatialDecimals is None:
                round_coords(agentCoords, p.spatialDecimals)

        def get_encounters():
            susceptibles = susceptible.nonzero()[0]
            indicateds = indicated.nonzero()[0]
            if not (len(susceptibles) and len(indicateds)):
                return []
            flip = len(susceptibles) < len(indicateds)
            if not flip:
                ids1, ids2 = susceptibles, indicateds
            else:
                ids1, ids2 = indicateds, susceptibles
            adjCoords = agentCoords - minCoords
            strategy = partial(
                accelerated_neighbours_radius_array,
                leafsize = 128
                )
            contacts = strategy(
                adjCoords[ids1],
                adjCoords[ids2],
                p.contactLength / 1000.,
                maxCoords - minCoords,
                )
            encounters = np.array([
                (id1, id2)
                    for id1, subcontacts in zip(ids2, contacts)
                        for id2 in ids1[subcontacts]
                ]) # <- very quick
            if len(encounters) and flip:
                encounters = encounters[:, slice(None, None, -1)]
            return encounters

        def get_newIndicateds(rng):
            encounters = get_encounters()
            if len(encounters):
                return np.unique(
                    encounters[
                        rng.random(encounters.shape[0]) < p.infectionChance
                        ][:, 1]
                    )
            else:
                return []

        def iterate_indicateds(rng):
            newIndicateds = get_newIndicateds(rng)
            indicated[newIndicateds] = True
            susceptible[newIndicateds] = False

        def iterate_recovered(rng):
            indicateds = indicated.nonzero()[0]
            recovery = rng.normal(
                p.recoverMean,
                p.recoverSpread,
                len(indicateds),
                ) < timeIndicated[indicated]
            indicated[indicateds] = ~recovery
            recovered[indicateds] = recovery

        def get_rng(addseed = None):
            if addseed is None:
                addseed = int(agentCoords.sum())
            seed = int(p.seed + addseed)
            return np.random.default_rng(seed)

        def add_mystery_indicateds(rng):
            nonSusceptible = susceptible.nonzero()[0]
            nNew = min(len(nonSusceptible), p.initialIndicated)
            newCases = rng.choice(nonSusceptible, nNew, replace = False)
            indicated[newCases] = True
            susceptible[newCases] = False

        def randomise_coords(rng):
            warnings.warn(
                "Setting coords randomly - did you expect this?",
                stacklevel = 3
                )
            agentCoords[...] = \
                rng.random((nAgents, 2)) \
                * (maxCoords - minCoords) \
                + p.corner

        def randomise_headings(rng):
            warnings.warn(
                "Setting headings randomly - did you expect this?",
                stacklevel = 3
                )
            headings[...] = rng.random(nAgents) * 2. * np.pi

        def initialise():
            rng = get_rng(0)
            if np.isnan(agentCoords).any():
                randomise_coords(rng)
            if np.isnan(headings).any():
                randomise_headings(rng)
            update_statuses()
            if p.initialIndicated:
                add_mystery_indicateds(rng)

        def iterate():
            rng = get_rng()
            timeIndicated[~susceptible] += p.timescale
            iterate_coords(rng)
            iterate_indicateds(rng)
            iterate_recovered(rng)
            return p.timescale

        def stop():
            return not bool(len(indicated.nonzero()[0]))

        def update_statuses():
            indicated[recovered] = False
            susceptible[...] = True
            susceptible[indicated] = False
            susceptible[recovered] = False

        def _update():
            update_statuses()

        return locals()

In [ ]:
mymodel = Covid1()

In [ ]:
from everest.cascade import Inputs

In [ ]:
class Grouper:
    def __init__(self, **kwargs):
        for k, v in kwargs.items():
            setattr(self, k, v)

In [ ]:
mymodel = Covid1._construct(Grouper(
    # params
    aspect = 1.2, # x length relative to y length
    scale = 2., # y length in km
    corner = [0., 0.], # coords of bottom-left corner
    timescale = 1., # days per timestep
    popDensity = 508, # people per sq km
    initialIndicated = 1, # initial mystery cases
    directionChange = 0.5, # where 1 == 180 deg per day
    speed = 5, # agent travel speed in km / h
    infectionChance = 0.1, # chance of transmission by 'contact'
    recoverMean = 14, # average recovery time in days
    recoverSpread = 2, # standard deviations of recovery curve
    contactLength = 1.5, # proximity in metres defining 'contact'
    spatialDecimals = None, # spatial precision limit
    seed = 1066, # random seed
    # _configs
    agentCoords = None,
    headings = None,
    indicated = False,
    recovered = False,
    timeIndicated = 0.,
    ))

In [ ]:
mymodel['initialise']()

In [ ]:
iterate = mymodel['iterate']

In [ ]:
from timeit import timeit

In [ ]:
timeit(iterate, number = 100)

In [ ]:
mymodel['nAgents']

In [ ]:
3 / 0.08

In [ ]:
mymodel.

In [ ]:
myconstruct = mymodel._construct(
    )